# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [ ]:
# Y_RealCase_train = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',')
# df_new = Y_RealCase_train.drop(Y_RealCase_train.columns[-18:], axis=1)
# Y_data_train = df_new.drop(df_new.columns[17], axis=1)

In [ ]:
# np.savetxt("Y_RealCase_train.csv", Y_data_train.values, delimiter=",", fmt="%.8f")

In [2]:
X_train = pd.read_csv('RealCase/RealCase_X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('RealCase/RealCase_X_test.csv', header=None, delimiter=',').values

Y_train_pca = pd.read_csv('RealCase/RealCase_Y_train_pca.csv', header=None, delimiter=',').values
Y_test_pca = pd.read_csv('RealCase/RealCase_Y_test_pca.csv', header=None, delimiter=',').values
Realcase_data_pca = pd.read_csv('RealCase/RealCase_Y_pca.csv', header=None, delimiter=',').values

Y_train = pd.read_csv('RealCase/RealCase_Y_train_std.csv', header=None, delimiter=',').values
Y_test = pd.read_csv('RealCase/RealCase_Y_test_std.csv', header=None, delimiter=',').values
Realcase_data = pd.read_csv('RealCase/RealCase_Y.csv', header=None, delimiter=',').values



In [3]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_pca = torch.tensor(Y_train_pca, dtype=torch.float32)
test_y_pca = torch.tensor(Y_test_pca, dtype=torch.float32)
realcase_y_pca = torch.tensor(Realcase_data_pca, dtype=torch.float32)


torch.set_default_dtype(torch.float32)

# Emulator

In [4]:
Device = 'cpu'

## MVGP

In [5]:
checkpoint = torch.load('multitask_gp_checkpoint_Realcase_pca.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_pca, 
                                               num_latents=model_params['num_latents'],
                                               num_inducing=model_params['num_inducing'],  
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_pca.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()


C:\Users\r4718\AppData\Local\Temp\ipykernel_28836\3563098655.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('multitask_gp_checkpoint_Realcase_pc

MultitaskGaussianLikelihood(
  (raw_task_noises_constraint): GreaterThan(1.000E-04)
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

# PE

In [6]:
input_point = realcase_y_pca[1,:]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_pca, k=100)

bounds = bound.get_bounds(local_train_x)

In [8]:
row_idx = 1

estimated_params_tmp, Loss_tmp = Estimation.multi_start_estimation(
        MVGP_models, MVGP_likelihoods, row_idx, realcase_y_pca, bounds,
        Estimation.estimate_params_for_one_model_Adam, num_starts=16, num_iterations=2000, lr=0.01,
        patience=15, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
    )

In [9]:
estimated_params_tmp

array([2.087089  , 0.1603    , 0.451373  , 0.8825183 , 0.7017178 ,
       0.94163984, 0.28515026, 0.64933985, 0.1036    , 2.1799886 ],
      dtype=float32)

In [10]:
Loss_tmp

7.233599662780762

In [11]:
Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, torch.tensor(estimated_params_tmp).unsqueeze(0))

tensor([ 4.8996e+00, -1.6440e+01, -4.9566e-02, -2.2767e+00, -3.8035e+00,
        -2.3900e+00, -9.0779e-01, -9.4588e-01,  5.2678e-02,  2.2119e-01,
         1.1240e-01,  4.8254e-01,  1.3929e-01,  1.0623e+00, -3.1325e-01,
         1.1797e-02, -2.7062e-02, -2.1244e-01], grad_fn=<ViewBackward0>)

In [12]:
input_point

tensor([  3.5505, -16.0766,  -0.2143,  -2.0070,  -3.5602,  -2.7620,  -2.0801,
         -2.8424,  -0.2216,  -1.3187,   2.7081,   4.0316,  -0.4541,   3.0037,
         -1.8883,  -0.2389,   3.3049,  -2.5702])

In [ ]:
row_idx = 0

input_point = test_y_pca[row_idx,:]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_pca, k=100)

bounds = bound.get_bounds(local_train_x)

estimated_params_tmp, Loss_tmp = Estimation.multi_start_estimation(
        MVGP_models, MVGP_likelihoods, row_idx, test_y_pca, bounds,
        Estimation.estimate_params_for_one_model_Adam, num_starts=6, num_iterations=2000, lr=0.01,
        patience=15, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
    )

In [ ]:
estimated_params_tmp

In [ ]:
Loss_tmp

In [ ]:
test_x[row_idx,:]

In [ ]:
mcmc_result_Uniform = Estimation.run_mcmc_Uniform(
        Prediction.preds_distribution, MVGP_models, MVGP_likelihoods, 
        row_idx, test_y, bounds, 
        num_sampling=1200, warmup_step=300, num_chains=1, device=Device
    )